In [1]:
import pandas as pd
from bioc import biocxml

from knowledge_engine.models.ontology_match import OntologyMatch
from knowledge_engine.models.stored_relationship import StoredRelationship
from notebooks.processing_utils import parse_document
from typing import Dict

with open('../data/BioRED/Train.BioC.XML', 'r') as bio_red_file:
    collection = biocxml.load(bio_red_file)

In [2]:
processed_dataset = pd.DataFrame([parse_document(doc) for doc in collection.documents])

In [5]:
processed_dataset.loc[0, 'relations']

[StoredRelationship(subject=StoredEntity(text='Hepatocyte nuclear factor-6', canonical_name='Hepatocyte nuclear factor-6', ontology_id=None, biolink_type='GeneOrGeneProduct', confidence=1.0), predicate='Association', object=StoredEntity(text='type II diabetes', canonical_name='type II diabetes', ontology_id=None, biolink_type='DiseaseOrPhenotypicFeature', confidence=1.0), confidence=1, evidence=''),
 StoredRelationship(subject=StoredEntity(text='glucose', canonical_name='glucose', ontology_id=None, biolink_type='ChemicalEntity', confidence=1.0), predicate='Positive_Correlation', object=StoredEntity(text='insulin', canonical_name='insulin', ontology_id=None, biolink_type='GeneOrGeneProduct', confidence=1.0), confidence=1, evidence=''),
 StoredRelationship(subject=StoredEntity(text='glucose', canonical_name='glucose', ontology_id=None, biolink_type='ChemicalEntity', confidence=1.0), predicate='Association', object=StoredEntity(text='type II diabetes', canonical_name='type II diabetes', o

In [9]:
from pathlib import Path
from knowledge_engine import ontology_manager

from deeponto.onto import Ontology

ontologies = {
    "omim": Ontology('./data/omim-ordo/omim.owl'),
    "ordo": Ontology('./data/omim-ordo/ordo.owl')
}


Please enter the maximum memory located to JVM [8g]:
2025-08-21 17:00:08,482 - deeponto - INFO - 1024M maximum memory allocated to JVM.
2025-08-21 17:00:08,482 - deeponto - INFO - JVM started successfully.


In [43]:
from importlib import reload
import ontology_utils
reload(ontology_utils)

def search_ontology(lookup: str, ontologies: dict[str, Ontology]):
    lookup = lookup.lower()
    for ontology in ontologies.keys():
        for iri, term in ontologies['omim'].owl_classes.items():
            annotations = ontologies['omim'].get_annotations(term)
            if lookup in annotations:
                yield ontology, iri, annotations, ontology_utils.get_all_relationships_for_entity(term, ontologies[ontology])

next(search_ontology('aarskog syndrome, autosomal dominant', ontologies))

Error getting annotation relationships: 'Cannot retrieve unknown IRI: http://www.w3.org/2000/01/rdf-schema#comment.'


('omim',
 'http://omim.org/entry/100050',
 ['aarskog syndrome, autosomal dominant'],
 {'hierarchical': [],
  'outgoing': [],
  'incoming': [],
  'annotations': [],
  'equivalences': []})

In [46]:
with open('../data/BioRED/Train.BioC.XML', 'r') as bio_red_file:
    collection = biocxml.load(bio_red_file)

In [50]:
processed_dataset.loc[0, 'relations'][0]

StoredRelationship(subject=StoredEntity(text='Hepatocyte nuclear factor-6', canonical_name='Hepatocyte nuclear factor-6', ontology_id=None, biolink_type='GeneOrGeneProduct', confidence=1.0), predicate='Association', object=StoredEntity(text='type II diabetes', canonical_name='type II diabetes', ontology_id=None, biolink_type='DiseaseOrPhenotypicFeature', confidence=1.0), confidence=1, evidence='')

In [53]:
from neo4j import GraphDatabase

In [54]:
gd = GraphDatabase()

In [55]:
from processing_utils import Neo4jBiolinkLoader, load_multiple_documents

In [ ]:
neo4j_config = {
        'neo4j_uri': 'bolt://localhost:7687',
        'neo4j_user': 'neo4j',
        'neo4j_password': 'test12345'
    }

# Or load multiple documents
document_batches = [
        row[1] for row in processed_dataset.iterrows()
        # Add more document batches...
]

load_multiple_documents(document_batches, **neo4j_config)